In [2]:
import pandas as pd

In [12]:
doctors = pd.read_csv('doctors_data_clean.csv')
doctors.head()


,cmp,apellidos,nombres,apellido_paterno,apellido_materno,status,foto_url,specialty,specialty_id,tipo,codigo
0,21,Berrios Reiterer,Jorge Enrique,Berrios,Reiterer,HÁBIL,fotos/00021.jpg,[Gastroenterología|Medicina Interna],[29],[RNE],[003319|001650]
1,22,Robinson Gunther,Fernando,Robinson,Gunther,HÁBIL,fotos/00022.jpg,[Radiología],[46],[RNE],[000119]
2,33,Abugattas Jasaui,Ricardo Jorge,Abugattas,Jasaui,HÁBIL,fotos/00033.jpg,[Cardiología],[8],[RNE],[000676]
3,57,Jara Velarde,Juan Alfredo,Jara,Velarde,HÁBIL,fotos/00057.jpg,[Neurología],[36],[RNE],[001540]
4,69,Giro Palau,Elvira Rosa,Giro,Palau,HÁBIL,fotos/00069.jpg,[Medicina De Rehabilitación],[121],[RNE],[001063]


In [49]:
# Create specialty codes
specialty_codes = doctors.copy()
specialty_codes = specialty_codes[['specialty', 'specialty_id']]
specialty_codes['specialty_arr'] = specialty_codes['specialty'].apply(
    lambda x: str(x).strip('[]').replace('|', ',').split(',') if pd.notnull(x) else []
)

specialty_codes['specialty_arr_count'] = specialty_codes['specialty_arr'].apply(len)
specialty_codes = specialty_codes[specialty_codes['specialty_arr_count'] == 1]
specialty_codes['specialty_id_real'] = specialty_codes['specialty_id'].apply(
    lambda x: int(str(x).strip('[]').replace('|', ',').split(',')[0]) if pd.notnull(x) and str(x).strip('[]') else None
)
specialty_codes_unique = specialty_codes.drop_duplicates(subset=['specialty_id_real'])
specialty_codes_unique['specialty_name'] = specialty_codes_unique['specialty_arr'].apply(
    lambda x: x[0] if pd.notnull(x) and len(x) > 0 else None
)


specialty_codes_unique = specialty_codes_unique.drop(columns=['specialty', 'specialty_id','specialty_arr','specialty_arr_count'])
specialty_codes_unique.sort_values(by='specialty_id_real', inplace=True)
specialty_codes_unique.to_csv('specialty_codes_unique.csv', index=False)
# Create a new column with the specialty IDs as a list


/var/folders/bn/4tgvjk0x4xsc_2hg3l4q35mm0000gn/T/ipykernel_84107/3231010062.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  specialty_codes_unique['specialty_name'] = specialty_codes_unique['specialty_arr'].apply(


In [53]:
doctors_clean = doctors.rename(columns={'cmp': 'CMP'})  # Do not use inplace to get a new DataFrame

# Drop columns that are not needed
doctors_clean.drop(columns=['apellidos'], inplace=True)

# Filter doctors not available
doctors_clean = doctors_clean[doctors_clean['status'] == 'HÁBIL']

# Create a new column with the specialty IDs as a list
doctors_clean['specialty_id_arr'] = doctors_clean['specialty_id'].apply(
    lambda x: str(x).strip('[]').replace('|', ',').split(',') if pd.notnull(x) else []
)

In [67]:


# doctors_clean['specialty_id_real'] = doctors_clean['specialty_id'].apply(
#     lambda x: int(str(x).strip('[]').replace('|', ',').split(',')[0]) if pd.notnull(x) and str(x).strip('[]') else None
# )
# doctors_clean =doctors_clean.drop(columns=['specialty','specialty_id','tipo','codigo','specialty_id_arr' ])
doctors_clean = doctors_clean.rename(columns={'specialty_id_real': 'especialidad_id'})
doctors_clean.to_csv('doctors_clean.csv', index=False)